In [1]:
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000


In [2]:
import os
files_path = [os.path.join(r,file) for r,d,f in os.walk(r"C:\Users\Peter\Desktop\Springboard\capstone1\Bought hands\unzipped") for file in f]
#print(files_path)

from IPython.core.interactiveshell import InteractiveShell # for all output 
InteractiveShell.ast_node_interactivity = "all"


In [3]:
import pandas as pd
import numpy as np


#Functions:

def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]




rows_list=[]
for f in files_path:    
    file1 = open(f, 'r') 
    Lines = file1.readlines() 
    for index,i in enumerate(Lines): 
        try:
            if left(i.strip(),5) == '#Game' :
                column = 0

                blind = Lines[index+5]
                game_number = int(right(i.strip(),9)) #got game number
        except:
            pass

    a = list()
    for index,i in enumerate(Lines): 
        if left(i.strip(),5) == '#Game' :
            a.append(index)
            while left(i.strip(),5) != '#Game':
                a.append(i)
                index+=1
            # here a is a list of indeces

    
    
# fix this loop!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#this loop contains each game as a nested array

    games= list()
    for ind, value in enumerate(a):
        try:
            if ind<=len(a):
            #if ind <1: #we only get first few and the last one
                games.append(Lines[a[ind-1]:a[ind]]) #tab it if we add the loop above
        #print(games)
        #len(games)
        except:
            pass
    #games.append(Lines[a[-1]:]) #get last line 


    
#this is a very long loop and I gather all the data in it
    for index, game in enumerate(games):
        try:
            game_number = left(right(game[0],10),9)    
            small_blind_string_index = game[8].find('posts small blind')
            big_blind_string_index = game[9].find('posts big blind')    
            small_blind_name = game[8][0:small_blind_string_index] #with a trailing space
            big_blind_name = game[9][0:big_blind_string_index]
            i = 10 #this is where line **Dealing down cards ** is
            preflop=[]
            flop=[]
            turn=[]
            river=[]
            preflop_counter = 0
            flop_counter=0
            stack_sb_index = game[6].find('$')
            stack_bb_index = game[7].find('$')
            stack_sb = game[6][stack_sb_index+1:-3]
            stack_bb = game[7][stack_bb_index+1:-3]

            
            
            
#Getting preflop actions

            while not ((game[i][:13] == ('** Summary **')) or (game[i][0:18] == ('** Dealing flop **'))):


                if 'folds' in game[i]:
                    thistuple= ('f',0)
                    preflop.append(thistuple)
                if ('calls' in game[i]) or ('bets' in game[i]) or ('raises' in game[i]):
                    dollar = game[i].index('$')
                    amount = game[i][dollar+1:-2]
                    first_space = game[i].index(' ')
                    action = game[i][first_space+1:first_space+5]
                    tuple_to_add = (action,amount)
                    preflop.append(tuple_to_add)
                if 'checks' in game[i]:
                    thistuple= ('check',0)
                    preflop.append(thistuple)
                i=i+1
                preflop_counter +=1




#find indeces of flop, turn , river and summary

            street_indeces = []
            for index_in_game, line in enumerate(game):
                if '** Dealing flop' in game[index_in_game]:
                    for index_in_game, line in enumerate(game):
                        if line[0:15]=='** Dealing flop':
                            street_indeces.append(index_in_game)
                        if line[0:15]=='** Dealing turn':
                            street_indeces.append(index_in_game)
                        if line[0:16]=='** Dealing river':
                            street_indeces.append(index_in_game)
                if line[0:13]=='** Summary **':
                    street_indeces.append(index_in_game)
            
            
            
            
#Getting hole Cards            
            hole_cards1=[]
            hole_cards2=[]
            #get hole cards
            if  'shows' in str(game[street_indeces[-1]+1]):
                hole_cards1 =  game[street_indeces[-1]+1][-11:-1]
                hole_cards2 =  game[street_indeces[-1]+2][-11:-1]
            #now we need to assign the hole cards correctly
            hole_cards1_player = game[street_indeces[-1]+1][0:game[street_indeces[-1]+1].index(' ')+1]
            if hole_cards1_player== small_blind_name:
                sb_hole_cards=hole_cards1
                bb_hole_cards=hole_cards2
            else:
                sb_hole_cards=hole_cards2
                bb_hole_cards=hole_cards1
                
                
                
                
#Getting flop cards and actions
            flop_cards=[]
            turn_card=[]
            river_card=[]
            
            if len(street_indeces)>1:
                flop_cards = [game[street_indeces[0]][-13:-11],game[street_indeces[0]][-9:-7],game[street_indeces[0]][-5:-3]]
                for i in range(street_indeces[0],street_indeces[1]): #flop
                    if 'folds' in game[i]:
                        thistuple= ('f',0)
                        flop.append(thistuple)
                    if ('calls' in game[i]) or ('bets' in game[i]) or ('raises' in game[i]):
                        dollar = game[i].index('$')
                        amount = game[i][dollar+1:-2]
                        first_space = game[i].index(' ')
                        action = game[i][first_space+1:first_space+5]
                        tuple_to_add = (action,amount)
                        flop.append(tuple_to_add)
                    if 'checks' in game[i]:
                        thistuple= ('check',0)
                        flop.append(thistuple)
                    i=i+1

                    
                    
#Getting turn cards and actions
            if len(street_indeces)>2:
                turn_card = game[street_indeces[1]][-5:-3]
                for i in range(street_indeces[1],street_indeces[2]): #TURN
                    if 'folds' in game[i]:
                        thistuple= ('f',0)
                        turn.append(thistuple)
                    if ('calls' in game[i]) or ('bets' in game[i]) or ('raises' in game[i]):
                        dollar = game[i].index('$')
                        amount = game[i][dollar+1:-2]
                        first_space = game[i].index(' ')
                        action = game[i][first_space+1:first_space+5]
                        tuple_to_add = (action,amount)
                        turn.append(tuple_to_add)
                    if 'checks' in game[i]:
                        thistuple= ('check',0)
                        turn.append(thistuple)
                    i=i+1 
                    
                    
                    
#Getting river cards and actions
            if len(street_indeces)>3:
                river_card = game[street_indeces[2]][-5:-3]
                for i in range(street_indeces[2],street_indeces[3]):
                    if 'folds' in game[i]:
                        thistuple= ('f',0)
                        river.append(thistuple)
                    if ('calls' in game[i]) or ('bets' in game[i]) or ('raises' in game[i]):
                        dollar = game[i].index('$')
                        amount = game[i][dollar+1:-2]
                        first_space = game[i].index(' ')
                        action = game[i][first_space+1:first_space+5]
                        tuple_to_add = (action,amount)
                        river.append(tuple_to_add)
                    if 'checks' in game[i]:
                        thistuple= ('check',0)
                        river.append(thistuple)
                    i=i+1
                
#Create a dictionary with all the data per game
            game_row  = {'Game ID':game_number,'Player SB':small_blind_name,'Player BB':big_blind_name, 'Preflop actions':preflop, 'Flop actions':flop, 'Turn actions':turn,'River actions':river, 'Flop':flop_cards, 'Turn':turn_card, 'River':river_card, 'SB stack':stack_sb, 'BB stack': stack_bb, 'SB cards': sb_hole_cards, 'BB cards':bb_hole_cards}
            rows_list.append(game_row)

        except:
            pass




In [4]:
df = pd.DataFrame(rows_list)
df.set_index('Game ID')
df.head()

,Player SB,Player BB,Preflop actions,Flop actions,Turn actions,River actions,Flop,Turn,River,SB stack,BB stack,SB cards,BB cards
Game ID,,,,,,,,,,,,,
502874582,Oracool1,sirstadiljus,"[(f, 0)]",[],[],[],[],[],[],1.22,2.01,[],[]
603604223,ArcticWin,Ceaban,"[(call, 0.01), (check, 0)]","[(check, 0), (check, 0)]","[(check, 0), (check, 0)]","[(bets, 0.04), (call, 0.04)]","[5c, 9c, 5s]",7h,2c,2,1,"[ Qh, Tc ]","[ 6h, 2h ]"
603604231,Ceaban,ArcticWin,"[(f, 0)]",[],[],[],[],[],[],1.94,1.06,[],[]
603604236,ArcticWin,Ceaban,"[(f, 0)]",[],[],[],[],[],[],1.95,1.05,[],[]
603604241,Ceaban,ArcticWin,"[(f, 0)]",[],[],[],[],[],[],1.94,1.06,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
553224385,Sensei.Dbb,roghnov,"[(call, 0.01), (check, 0)]","[(check, 0), (bets, 0.02), (f, 0)]",[],[],"[Kc, 5d, 5s]",[],[],1.02,0.98,[],[]
553224389,roghnov,Sensei.Dbb,"[(rais, 0.03), (f, 0)]",[],[],[],[],[],[],1,1,[],[]
553224821,mikvell,John_Efapi,"[(call, 0.01), (check, 0)]","[(bets, 0.04), (call, 0.04)]","[(check, 0), (bets, 0.08), (f, 0)]",[],"[9h, 6h, 7h]",Ks,[],3.36,1.90,[],[]


,Game ID,Player SB,Player BB,Preflop actions,Flop actions,Turn actions,River actions,Flop,Turn,River,SB stack,BB stack,SB cards,BB cards
0,502874582,Oracool1,sirstadiljus,"[(f, 0)]",[],[],[],[],[],[],1.22,2.01,[],[]
1,603604223,ArcticWin,Ceaban,"[(call, 0.01), (check, 0)]","[(check, 0), (check, 0)]","[(check, 0), (check, 0)]","[(bets, 0.04), (call, 0.04)]","[5c, 9c, 5s]",7h,2c,2,1,"[ Qh, Tc ]","[ 6h, 2h ]"
2,603604231,Ceaban,ArcticWin,"[(f, 0)]",[],[],[],[],[],[],1.94,1.06,[],[]
3,603604236,ArcticWin,Ceaban,"[(f, 0)]",[],[],[],[],[],[],1.95,1.05,[],[]
4,603604241,Ceaban,ArcticWin,"[(f, 0)]",[],[],[],[],[],[],1.94,1.06,[],[]


Data wrangling is over

In [5]:
print(df['SB cards'])

0                 []
1         [ Qh, Tc ]
2                 []
3                 []
4                 []
             ...    
520801            []
520802            []
520803            []
520804    [ 3d, 3s ]
520805            []
Name: SB cards, Length: 520806, dtype: object


In [6]:
print(game_row)

{'Game ID': '553224845', 'Player SB': 'mikvell ', 'Player BB': 'John_Efapi ', 'Preflop actions': [('call', '0.01'), ('check', 0)], 'Flop actions': [('check', 0), ('check', 0)], 'Turn actions': [('check', 0), ('check', 0)], 'River actions': [('bets', '0.06'), ('f', 0)], 'Flop': ['8s', 'Td', 'Tc'], 'Turn': '4c', 'River': 'Jh', 'SB stack': '3.48', 'BB stack': '1.78', 'SB cards': [], 'BB cards': []}


In [7]:
df.tail()

,Game ID,Player SB,Player BB,Preflop actions,Flop actions,Turn actions,River actions,Flop,Turn,River,SB stack,BB stack,SB cards,BB cards
520801,553224385,Sensei.Dbb,roghnov,"[(call, 0.01), (check, 0)]","[(check, 0), (bets, 0.02), (f, 0)]",[],[],"[Kc, 5d, 5s]",[],[],1.02,0.98,[],[]
520802,553224389,roghnov,Sensei.Dbb,"[(rais, 0.03), (f, 0)]",[],[],[],[],[],[],1,1,[],[]
520803,553224821,mikvell,John_Efapi,"[(call, 0.01), (check, 0)]","[(bets, 0.04), (call, 0.04)]","[(check, 0), (bets, 0.08), (f, 0)]",[],"[9h, 6h, 7h]",Ks,[],3.36,1.90,[],[]
520804,553224834,John_Efapi,mikvell,"[(rais, 0.05), (call, 0.04)]","[(check, 0), (check, 0)]","[(check, 0), (check, 0)]","[(check, 0), (check, 0)]","[9d, Qc, 5d]",7d,Ac,3.42,1.84,"[ 3d, 3s ]","[ 4h, Ah ]"
520805,553224845,mikvell,John_Efapi,"[(call, 0.01), (check, 0)]","[(check, 0), (check, 0)]","[(check, 0), (check, 0)]","[(bets, 0.06), (f, 0)]","[8s, Td, Tc]",4c,Jh,3.48,1.78,[],[]


In [8]:
len(rows_list)

520806

In [9]:
print(df[55:56]) # ** Dealing donw cards



      Game ID   Player SB                 Player BB Preflop actions  \
55  502896851  Pbl6aJIKa   ** Dealing down cards **        [(f, 0)]   

   Flop actions Turn actions River actions Flop Turn River SB stack BB stack  \
55           []           []            []   []   []    []        2     2.02   

   SB cards BB cards  
55       []       []  


In [10]:
missed_hands = df[df['Player BB']== '** Dealing down cards **']
missed_hands.count()

Game ID            17640
Player SB          17640
Player BB          17640
Preflop actions    17640
Flop actions       17640
Turn actions       17640
River actions      17640
Flop               17640
Turn               17640
River              17640
SB stack           17640
BB stack           17640
SB cards           17640
BB cards           17640
dtype: int64

In [11]:
no_missed_hands = df[df['Player BB']!= '** Dealing down cards **']
no_missed_hands.count()

Game ID            503166
Player SB          503166
Player BB          503166
Preflop actions    503166
Flop actions       503166
Turn actions       503166
River actions      503166
Flop               503166
Turn               503166
River              503166
SB stack           503166
BB stack           503166
SB cards           503166
BB cards           503166
dtype: int64